<a href="https://colab.research.google.com/github/yukinaga/ai_music/blob/main/section_4/01_multitrack_musicvae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MusicVAEによる作曲
「MusicVAE」により、複数のトラックからなる曲を生成します。  
生成には時間がかかるので、「編集」→「ノートブックの設定」→「ハードウェア アクセラレータ」で「GPU」を選択しておきましょう。  
このノートブックのコードは、以下のリンク先のコードを参考にしています。  
https://g.co/magenta/musicvae-colab

## ライブラリのインストールとモデルのダウンロード
Magentaと共に、音楽生成用のライブラリpyFluidSynth、MIDIデータを処理するためのpretty_midiなどをインストールします。  
また、MusicVAEのモデルをダウンロードします。

In [ ]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta

!gsutil -q -m cp gs://download.magenta.tensorflow.org/models/music_vae/multitrack/* /content/

## ライブラリの導入
Magentaの必要な機能と、NumPyなどのライブラリを導入します。

In [ ]:
import numpy as np
from google.colab import files

import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
from magenta.music.sequences_lib import concatenate_sequences

import note_seq

## 各設定値
曲の生成に関する各値を設定します。

In [ ]:
BATCH_SIZE = 4  # 一度に扱うデータ数
Z_SIZE = 512  # 潜在変数の数
TOTAL_STEPS = 512  # コードのベクトル化に使用
CHORD_DEPTH = 49  # コードのベクトル化に使用
SEQ_TIME = 2.0  # 各NoteSequenceの長さ

## 関数の設定
頻繁に行う処理を、関数にまとめておきます。

In [ ]:
def trim(seqs, seq_time=SEQ_TIME):  # NoteSequenceの長さを揃える
    for i in range(len(seqs)):
        seqs[i] = mm.extract_subsequence(seqs[i], 0.0, seq_time)
        seqs[i].total_time = seq_time

def encode_chord(chord):  # コードの文字列をベクトルに変換
    index = mm.TriadChordOneHotEncoding().encode_event(chord)
    encoded = np.zeros([TOTAL_STEPS, CHORD_DEPTH])
    encoded[0,0] = 1.0
    encoded[1:,index] = 1.0
    return encoded

def set_instruments(note_sequences):  # 楽器の調整
    for i in range(len(note_sequences)):
        for note in note_sequences[i].notes:
            if note.is_drum:
                note.instrument = 9

## Conditionalではないモデル
Conditionalではない、通常のVAEの学習済みモデルをチェックポイントとして読み込みます。

In [ ]:
config = configs.CONFIG_MAP["hier-multiperf_vel_1bar_med"]
model = TrainedModel(
    config,
    batch_size=BATCH_SIZE,
    checkpoint_dir_or_path="/content/model_fb256.ckpt")

モデルからランダムに音声をサンプリングします。  

In [ ]:
temperature = 0.2
seqs = model.sample(n=BATCH_SIZE, length=TOTAL_STEPS, temperature=temperature)

trim(seqs)
for seq in seqs:
    mm.play_sequence(seq, synth=mm.fluidsynth)

潜在変数から曲を生成します。  
連続的に変化する潜在変数を、デコードしてつなげることにより曲を生成します。

In [ ]:
num_bars = 32
temperature = 0.2

z1 = np.random.normal(size=[Z_SIZE])
z2 = np.random.normal(size=[Z_SIZE])
z = np.array([z1+z2*t for t in np.linspace(0, 1, num_bars)])  # z1とz2の間を線形補間

seqs = model.decode(length=TOTAL_STEPS, z=z, temperature=temperature)

trim(seqs)
set_instruments(seqs)
seq = concatenate_sequences(seqs)

mm.plot_sequence(seq)
mm.play_sequence(seq, synth=mm.fluidsynth)

`NoteSequence`をMIDIデータに変換し、保存してダウンロードします。


In [ ]:
note_seq.sequence_proto_to_midi_file(seq, "unconditional_vae.mid")  #MIDI　データに変換し保存
files.download("unconditional_vae.mid")  # ダウンロード

## Conditionalなモデル
コードをラベルとしたConditional VAEのモデルを読み込みます。

In [ ]:
config = configs.CONFIG_MAP["hier-multiperf_vel_1bar_med_chords"]
model = TrainedModel(
    config,
    batch_size=BATCH_SIZE,
    checkpoint_dir_or_path="/content/model_chords_fb64.ckpt")

コードをラベルとして入力し、モデルからランダムに音声をサンプリングします。  

In [ ]:
chord = "C"
temperature = 0.2

seqs = model.sample(n=BATCH_SIZE, length=TOTAL_STEPS, temperature=temperature,
                    c_input=encode_chord(chord))

for seq in seqs:
    mm.play_sequence(seq, synth=mm.fluidsynth)

潜在変数から曲をデコードする際に、コードをラベルとして入力します。  
これにより、そのコードをベースにした曲のNoteSeqenceを生成することができます。

In [ ]:
chord_1 = "C"
chord_2 = "Caug"
chord_3 = "Am"
chord_4 = "E"
chords = [chord_1, chord_2, chord_3, chord_4]

temperature = 0.2
z = np.random.normal(size=[1, Z_SIZE])
seqs = [
    model.decode(length=TOTAL_STEPS,
                 z=z,
                 temperature=temperature,
                 c_input=encode_chord(c))[0]
    for c in chords
]

trim(seqs)
set_instruments(seqs)
seq = concatenate_sequences(seqs)

mm.plot_sequence(seq)
mm.play_sequence(seq, synth=mm.fluidsynth)

潜在変数とベースとなるコード進行から曲を生成します。  
連続的に変化する潜在変数を、コードとともにデコードしてつなげることにより曲を生成します。

In [ ]:
chord_1 = "Dm"
chord_2 = "F"
chord_3 = "Am"
chord_4 = "G"
chords = [chord_1, chord_2, chord_3, chord_4]

num_bars = 32
temperature = 0.2

z1 = np.random.normal(size=[Z_SIZE])
z2 = np.random.normal(size=[Z_SIZE])
z = np.array([z1+z2*t for t in np.linspace(0, 1, num_bars)])  # z1とz2の間を線形補間

seqs = [
    model.decode(
        length=TOTAL_STEPS,
        z=z[i:i+1, :],
        temperature=temperature,
        c_input=encode_chord(chords[i%4])
        )[0]
    for i in range(num_bars)
]

trim(seqs)
set_instruments(seqs)
seq = concatenate_sequences(seqs)

mm.plot_sequence(seq)
mm.play_sequence(seq, synth=mm.fluidsynth)

`NoteSequence`をMIDIデータに変換し、保存してダウンロードします。


In [ ]:
note_seq.sequence_proto_to_midi_file(seq, "conditional_vae.mid")  #MIDI　データに変換し保存
files.download("conditional_vae.mid")  # ダウンロード